In [30]:
from glob import glob
import numpy as np
from structs import read_map
import subprocess
import os

In [31]:
try:
    os.mkdir("obs/")
    os.mkdir("obs/rooms/")
    os.mkdir("obs/random/")
    os.mkdir("obs/mazes/")
except:
    pass

In [25]:
map_sources = [
    "../../hog2/maps/rooms/32room_*.map",
    #"../../hog2/maps/random/random512*.map",
    #"../../hog2/maps/mazes/maze512*.map"
]

def mapn_to_obsn(filepath):
    return filepath.replace("../../hog2/maps/", "obs/").replace(".map", "*.obs")
    

In [18]:
maps = []
for source in map_sources:
    maps += glob(source)

In [36]:
for m in maps:
    outfname = m.replace("../../hog2/maps/", "obs/").replace(".map", "_1000.obs")
    if not os.path.exists(outfname) or True:
        command = [
            "./genobs.py", m, "1000", "1", outfname
        ]
        subprocess.run(command)
        for i in ["0", "10", "100"]:
            command = "head -" + i + " " + outfname + " > " + outfname.replace("_1000.obs", "_" + i + ".obs")
            subprocess.run(command, shell = True)

In [42]:
with open("commands.run", "wt") as comf:
    allendt = "5000"
    for m in maps:
        grid = read_map(m)
        for i in range(0, 512):
            if grid.occupancy[i,i] == 0:
                start_loc = (str(i), str(i))
                break
        for i in range(511, -1, -1):
            if grid.occupancy[i,i] == 0:
                goal_loc = (str(i), str(i))
                break
        for obs in glob(mapn_to_obsn(m)):
            for search in ["sipp","pdap"]:
                command = ["../build_release/pdap",
                           "--search " + search,
                           "--obstacles " + obs,
                           "--map " + m,
                           "--startx " + start_loc[0],
                           "--starty " + start_loc[1],
                           "--goalx " + goal_loc[0],
                           "--goaly " + goal_loc[1],
                           "--startt 0",
                           "--allendt", allendt,
                           " > " + "results/" + "_".join([search, obs.split("/")[-1], m.split("/")[-1],
                                             start_loc[0], start_loc[1], goal_loc[0], goal_loc[1], 
                                             allendt, ".res"])
                          ]
                print(" ".join(command), file = comf)
    
    

In [39]:
command

'head -100 obs/mazes/maze512-32-6_1000.obs > obs/mazes/maze512-32-6_100.obs'